In [3]:
import torch
import torch.nn.functional as F
import numpy as np
# activation functions are included in torch.nn.functional
# but now deprecated, can use nn.sigmoid, nn.tanh directly
import matplotlib.pyplot as plt

In [207]:
class ThreeLayerNeuralNetwork:
    """
    3-NN 
    input=2
    tanh_layer input=2, output=4 (y1=tanh(x@w1+b1))
    sigmoid_layer  input=4, output=4 (y2=relu(y1@w2+b2))
    tanh_layer input=4, output=1 (y=sigmoid(y2@w3+b3))
    """
    def __init__(self, 
                 lr=5e-3, 
                 epoch=3000):
        self.w1 = torch.nn.Parameter(torch.randn(2,4))
        self.b1 = torch.nn.Parameter(torch.zeros(4))
        self.w2 = torch.nn.Parameter(torch.randn(4,4))
        self.b2 = torch.nn.Parameter(torch.zeros(4))
        self.w3 = torch.nn.Parameter(torch.randn(4,1))
        self.b3 = torch.nn.Parameter(torch.zeros(1))
        
        self.epoch = epoch
        
        self.optimizer = torch.optim.SGD([
            self.w1, self.b1, 
            self.w2, self.b2,
            self.w3, self.b3
        ], lr = lr)
        self.loss_func = torch.nn.BCEWithLogitsLoss()
        
    def network(self, x):
        y1 = F.tanh(torch.mm(x, self.w1) + self.b1)
        y2 = F.tanh(torch.mm(y1, self.w2) + self.b2)
        y = F.sigmoid(torch.mm(y2, self.w3) + self.b3)
        return y
    
    def train(self, x_train, y_train):
        for e in range(self.epoch):
            y_estimate = self.network(x_train)
            loss = self.loss_func(y_estimate, y_train) # attention: do not swap the position!
            loss.backward()
            self.optimizer.step()
            acc = self.acc(y_train, self.predict(x_train))
            if e%100==0 or e==self.epoch-1:
                print("epoch{}/{}, loss={}, acc={}".format(e, self.epoch, loss, acc))
            self.optimizer.zero_grad()
    
    def predict(self, x_pred):
        return self.network(x_pred).ge(0.5).float()
    
    def acc(self, y_train, y_pred):
        return (y_pred == y_train).float().sum() / y_train.shape[0]

In [208]:
def make_dataset(sample_count, input_dim=2, label_classes=2):
    x = np.zeros((sample_count, input_dim))
    y = np.zeros((sample_count, 1))
    N = int(sample_count/label_classes)
    for c in range(label_classes):
        ix = range(N*c,N*(c+1))
        t = np.linspace(c*3.12,(c+1)*3.12,N) + np.random.randn(N)*0.2
        r = 4*np.sin(4*t) + np.random.randn(N)*0.2
        x[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = c
    return x,y

In [209]:
train_x, train_y = make_dataset(10000)

In [210]:
threeNN = ThreeLayerNeuralNetwork()
threeNN.train(
    torch.Tensor(train_x),
    torch.Tensor(train_y),
)

/Users/qhu/Envrionments/miniforge3/lib/python3.9/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/Users/qhu/Envrionments/miniforge3/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch0/3000, loss=0.7428659200668335, acc=0.36970001459121704
epoch100/3000, loss=0.7325183749198914, acc=0.3921999931335449
epoch200/3000, loss=0.7211032509803772, acc=0.4611999988555908
epoch300/3000, loss=0.7096757292747498, acc=0.5450999736785889
epoch400/3000, loss=0.6995582580566406, acc=0.5952000021934509
epoch500/3000, loss=0.6914080381393433, acc=0.640999972820282
epoch600/3000, loss=0.6849150657653809, acc=0.6583999991416931
epoch700/3000, loss=0.6795336008071899, acc=0.6775000095367432
epoch800/3000, loss=0.6749041676521301, acc=0.6901999711990356
epoch900/3000, loss=0.6708290576934814, acc=0.7087000012397766
epoch1000/3000, loss=0.6671918034553528, acc=0.7124999761581421
epoch1100/3000, loss=0.6639143824577332, acc=0.7146000266075134
epoch1200/3000, loss=0.66093909740448, acc=0.7159000039100647
epoch1300/3000, loss=0.6582201719284058, acc=0.7170000076293945
epoch1400/3000, loss=0.6557191610336304, acc=0.7246000170707703
epoch1500/3000, loss=0.6534032225608826, acc=0.7361999